In [1]:
import os

In [2]:
%pwd

'c:\\Users\\GEN-AI\\End-to-End-machine-project-with-mlflow-\\research'

In [ ]:
#os.chdir("../")

In [4]:
%pwd

'c:\\Users\\GEN-AI\\End-to-End-machine-project-with-mlflow-'